In [3]:
import os
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import numpy as np
from transformers import pipeline
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer


pd.set_option("max_columns", 300)
pd.set_option('display.max_colwidth', None)
os.chdir(os.getcwd().replace('notebooks','').replace('medi_crawler',''))

In [4]:
import config as CONFIG


# Dataset

In [5]:
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'df' with features and labels
X = pd.DataFrame(pd.read_csv("data/medi_crawler/processed/labeled_data.csv")['Title'])
y = pd.DataFrame(pd.read_csv("data/medi_crawler/processed/labeled_data.csv")['label'])

bool_not_nan = ~X['Title'].isna()
X = X[bool_not_nan]
y = y[bool_not_nan]


# Model

In [6]:
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
MODEL_NAME = 'distilbert-base-uncased'

BATCH_SIZE = 8
N_EPOCHS = 30
MAX_LEN = 256

In [7]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert X_train and X_test to list of strings
X_train = X_train.squeeze().tolist()
X_test = X_test.squeeze().tolist()


In [8]:
# Define a tokenizer object
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
# Tokenize the text
train_encodings = tokenizer(X_train,
                            truncation=True,
                            padding=True,
                            max_length=MAX_LEN,
                            return_tensors='tf')  # Convert to TensorFlow tensors


test_encodings = tokenizer(X_test,
                           truncation=True,
                           padding=True,
                           max_length=MAX_LEN,
                           return_tensors='tf')  # Convert to TensorFlow tensors


In [9]:

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), list(y_train.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), list(y_test.values)))


In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                    list(y_train.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                    list(y_test.values)))

In [11]:
import tensorflow as tf

class Recall(tf.keras.metrics.Metric):
    def __init__(self, name='recall', **kwargs):
        super(Recall, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(tf.argmax(y_pred, axis=-1), tf.float32)
        true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred), tf.float32)
        false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred)), tf.float32)
        self.true_positives.assign_add(true_positives)
        self.false_negatives.assign_add(false_negatives)
    
    def result(self):
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return recall
    
    def reset_states(self):
        self.true_positives.assign(0.0)
        self.false_negatives.assign(0.0)


In [12]:
# Instantiate the model and tokenizer

model = DistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

# Choose the optimizer
optimizerr = torch.optim.AdamW(model.parameters(), lr=5e-5)


model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=[Recall()])
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/30
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
24/24 [==============================] - 80s 3s/step - loss: 0.6672 - recall: 0.0483
Epoch 2/30


c:\Users\Robert\Documents\Projekte\dev\statsfaction\.venv\lib\site-packages\keras\engine\training.py:2416: UserWarning: Metric Recall implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


24/24 [==============================] - 50s 2s/step - loss: 0.5584 - recall: 0.2650
Epoch 3/30
24/24 [==============================] - 49s 2s/step - loss: 0.4978 - recall: 0.4417
Epoch 4/30
24/24 [==============================] - 49s 2s/step - loss: 0.2256 - recall: 0.4533
Epoch 5/30
24/24 [==============================] - 50s 2s/step - loss: 0.1357 - recall: 0.4517
Epoch 6/30
24/24 [==============================] - 50s 2s/step - loss: 0.1069 - recall: 0.4617
Epoch 7/30
24/24 [==============================] - 51s 2s/step - loss: 0.0400 - recall: 0.4517
Epoch 8/30
24/24 [==============================] - 51s 2s/step - loss: 0.0449 - recall: 0.4583
Epoch 9/30
24/24 [==============================] - 50s 2s/step - loss: 0.0452 - recall: 0.5150
Epoch 10/30
24/24 [==============================] - 50s 2s/step - loss: 0.1612 - recall: 0.4117
Epoch 11/30
24/24 [==============================] - 50s 2s/step - loss: 0.0738 - recall: 0.5200
Epoch 12/30
24/24 [==============================

In [11]:
import pickle

# Save the model to a file
with open('recall_trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)


c:\Users\Robert\Documents\Projekte\dev\statsfaction\.venv\lib\site-packages\transformers\generation\tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...classifier
......vars
.........0
.........1
...distilbert
......vars
...distilbert\embeddings
......vars
.........0
.........1
...distilbert\embeddings\LayerNorm
......vars
.........0
.........1
...distilbert\embeddings\dropout
......vars
...distilbert\transformer
......vars
...distilbert\transformer\layer\tf_transformer_block
......vars
...distilbert\transformer\layer\tf_transformer_block\attention
......vars
...distilbert\transformer\layer\tf_transformer_block\attention\dropout
......vars
...distilbert\transformer\layer\tf_transformer_block\attention\k_lin
......vars
.........0
.........1
...distilbert\transformer\layer\tf_transformer_block\attention\out_lin
......vars
.........0
.........1
...distilbert\transformer\layer\tf_transformer_block\attention\q_lin
......vars
.........0
.........1
...distilbert\transformer\layer\tf_transformer_block\attention\v_lin
......vars
.........0
.........1
...distilbert\transformer

In [13]:

# Predict Test data
preds = model.predict(test_dataset).logits

preds = tf.nn.softmax(preds, axis=1).numpy()  

y_preds = np.round(preds[:,1],0)

from sklearn.metrics import classification_report

# Assuming you have the predicted labels 'y_pred' and the true labels 'y_true'
report = classification_report(y_test, y_preds)

print(report)


48/48 [==============================] - 6s 61ms/step
              precision    recall  f1-score   support

       False       0.96      0.66      0.78        35
        True       0.50      0.92      0.65        13

    accuracy                           0.73        48
   macro avg       0.73      0.79      0.71        48
weighted avg       0.83      0.73      0.74        48



### predict new unlabeled data

In [14]:
X_new_df = pd.DataFrame(pd.read_csv("data/medi_crawler/processed/unlabeled_data.csv")['Title'])
pmid_unlabeld = pd.DataFrame(pd.read_csv("data/medi_crawler/processed/unlabeled_data.csv")['pmid'])

unlabeled_not_nan = ~X_new_df['Title'].isna()

X_new_df = X_new_df[unlabeled_not_nan]
pmid_unlabeld = pmid_unlabeld[unlabeled_not_nan]

In [15]:
X_new = X_new_df.squeeze().values

encodings = tokenizer(X_new.squeeze().tolist(), 
                      max_length=MAX_LEN, 
                      truncation=True, 
                      padding=True)
# Transform to tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))

# Predict
preds = model.predict(dataset).logits

preds = tf.nn.softmax(preds, axis=1).numpy()  

4738/4738 [==============================] - 666s 140ms/step


In [16]:
output = pd.concat([X_new_df,pd.DataFrame(preds[:,1],index=X_new_df.index,columns=['pred'])],axis=1).sort_values('pred',ascending=False)
output.head()

,Title,pred
3056,Right ventricle to pulmonary artery conduit has a favorable impact on postoperative physiology after Stage I Norwood: preliminary results.,0.999816
2350,[Further development of pulmonary artery after Glenn procedure: effect of different antegrade pulmonary blood flows on cyanotic congenital heart defects].,0.999816
1864,Pulmonary artery size at the time of bidirectional cavopulmonary shunt and Fontan surgery influences long-term outcomes.,0.999816
3551,Changes in pulmonary artery size before and after total cavopulmonary connection.,0.999815
2058,Impact of shunt type on growth of pulmonary arteries after norwood stage I procedure: current best available evidence.,0.999815


In [ ]:
output.to_csv('data/medi_crawler/final/predict_label_title_recall.csv')

In [ ]:
def predict_proba(text_list, model, tokenizer):  
    #tokenize the text
    encodings = tokenizer(text_list, 
                          max_length=MAX_LEN, 
                          truncation=True, 
                          padding=True)
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = model.predict(dataset.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()      
    
    return res

predict_proba(strings_list[3], model, tokenizer)[:,1].mean()